<a href="https://colab.research.google.com/github/dmml-heriot-watt/group-coursework-ha/blob/harry-branch/Spotify_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the libraries**

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

**Loading in the dataset & removing unwanted columns**

In [71]:
df = pd.read_csv('top10s-raw.csv', encoding = 'latin-1')
df = df.iloc[:, 3:]

In [72]:
df.shape

(603, 12)

**Checking the different genre names**

In [73]:
df['top genre'].unique()

array(['neo mellow', 'detroit hip hop', 'dance pop', 'pop',
       'canadian pop', 'hip pop', 'barbadian pop', 'atl hip hop',
       'australian pop', 'indie pop', 'art pop', 'colombian pop',
       'big room', 'british soul', 'chicago rap', 'acoustic pop',
       'permanent wave', 'boy band', 'baroque pop', 'celtic rock',
       'electro', 'complextro', 'canadian hip hop', 'candy pop',
       'alaska indie', 'folk-pop', 'metropopolis', 'house',
       'australian hip hop', 'electropop', 'australian dance',
       'hollywood', 'canadian contemporary r&b',
       'irish singer-songwriter', 'tropical house', 'belgian edm',
       'french indie pop', 'hip hop', 'danish pop', 'latin',
       'canadian latin', 'electronic trap', 'edm', 'electro house',
       'downtempo', 'brostep', 'contemporary country', 'moroccan pop',
       'escape room', 'alternative r&b'], dtype=object)

**Checking the instances of each genre**

In [74]:
df['top genre'].value_counts()

dance pop                    327
pop                           60
canadian pop                  34
barbadian pop                 15
boy band                      15
electropop                    13
british soul                  11
big room                      10
canadian contemporary r&b      9
neo mellow                     9
art pop                        8
complextro                     6
hip pop                        6
australian dance               6
atl hip hop                    5
australian pop                 5
edm                            5
permanent wave                 4
latin                          4
hip hop                        4
colombian pop                  3
tropical house                 3
brostep                        2
downtempo                      2
electronic trap                2
belgian edm                    2
escape room                    2
folk-pop                       2
electro                        2
indie pop                      2
detroit hi

**Removing the space from the column name**

In [75]:
df.rename(columns={'top genre':'top_genre'}, inplace=True)

In [76]:
modified_df = df

**Combining similar genres into groups**

In [77]:
#adds all instances of pop in the genre column into one shared genre
for i in modified_df.top_genre:
    if 'pop' in i:
        modified_df.top_genre = modified_df.top_genre.replace(i, 'pop')

    elif 'hip hop' in i:
        modified_df.top_genre = modified_df.top_genre.replace(i, 'hip hop')

    elif 'edm' in i:
        modified_df.top_genre = modified_df.top_genre.replace(i, 'edm')

    elif 'r&b' in i:
        modified_df.top_genre = modified_df.top_genre.replace(i, 'pop')

    elif 'latin' in i:
        modified_df.top_genre = modified_df.top_genre.replace(i, 'latin')

    elif 'room' in i:
        modified_df.top_genre = modified_df.top_genre.replace(i, 'room')

    elif 'electro' in i:
        modified_df.top_genre = modified_df.top_genre.replace(i, 'edm')

#replaces all instances of chicago rap with hip hop
modified_df.top_genre = modified_df.top_genre.replace('chicago rap', 'hip hop')

**Checking the instances of different genres again**

In [78]:
modified_df.top_genre.value_counts()

pop                        495
boy band                    15
hip hop                     15
room                        12
edm                         12
british soul                11
neo mellow                   9
complextro                   6
australian dance             6
latin                        5
permanent wave               4
tropical house               3
brostep                      2
downtempo                    2
alaska indie                 1
irish singer-songwriter      1
hollywood                    1
house                        1
celtic rock                  1
contemporary country         1
Name: top_genre, dtype: int64

**Removing any columns with 3 or less instances**

In [79]:
cull_df = modified_df
value_counts = cull_df.stack().value_counts()
culled = value_counts[value_counts <= 3].index
cull_df.replace(culled, 'other', inplace=True)
cull_df.top_genre.value_counts()

pop                 495
hip hop              15
boy band             15
other                13
room                 12
edm                  12
british soul         11
neo mellow            9
complextro            6
australian dance      6
latin                 5
permanent wave        4
Name: top_genre, dtype: int64

**Showing that 'other' is present in the dataframe**

In [80]:
cull_df.head()

,top_genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,pop,2010,120,84,76,-3,29,71,200,10,14,80
3,pop,2010,other,92,70,-4,8,71,other,0,4,79
4,pop,2010,other,84,64,-5,9,43,221,2,4,78


**Removing all instances of 'other' and replcing it with the rounded mean value for that column**

In [81]:
mean_value = round(pd.to_numeric(cull_df['bpm'], errors='coerce').mean())
cull_df['bpm'] = cull_df['bpm'].replace('other', mean_value)

In [82]:
mean_value = round(pd.to_numeric(cull_df['nrgy'], errors='coerce').mean())
cull_df['nrgy'] = cull_df['nrgy'].replace('other', mean_value)

In [83]:
mean_value = round(pd.to_numeric(cull_df['dnce'], errors='coerce').mean())
cull_df['dnce'] = cull_df['dnce'].replace('other', mean_value)

In [84]:
mean_value = round(pd.to_numeric(cull_df['dB'], errors='coerce').mean())
cull_df['dB'] = cull_df['dB'].replace('other', mean_value)

In [85]:
mean_value = round(pd.to_numeric(cull_df['live'], errors='coerce').mean())
cull_df['live'] = cull_df['live'].replace('other', mean_value)

In [86]:
mean_value = round(pd.to_numeric(cull_df['val'], errors='coerce').mean())
cull_df['val'] = cull_df['val'].replace('other', mean_value)

In [87]:
mean_value = round(pd.to_numeric(cull_df['dur'], errors='coerce').mean())
cull_df['dur'] = cull_df['dur'].replace('other', mean_value)

In [88]:
mean_value = round(pd.to_numeric(cull_df['acous'], errors='coerce').mean())
cull_df['acous'] = cull_df['acous'].replace('other', mean_value)

In [89]:
mean_value = round(pd.to_numeric(cull_df['spch'], errors='coerce').mean())
cull_df['spch'] = cull_df['spch'].replace('other', mean_value)

In [90]:
mean_value = round(pd.to_numeric(cull_df['pop'], errors='coerce').mean())
cull_df['pop'] = cull_df['pop'].replace('other', mean_value)

**Showing that 'other' has now been replaced**

In [91]:
cull_df.head()

,top_genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,pop,2010,120,84,76,-3,29,71,200,10,14,80
3,pop,2010,116,92,70,-4,8,71,218,0,4,79
4,pop,2010,116,84,64,-5,9,43,221,2,4,78


**Saving dataframe as cleaned CSV for analysis**

In [93]:
cull_df.to_csv('top10s-clean.csv')